In [1]:
import pyarrow.parquet as pq
import duckdb
conn = duckdb.connect()

In [4]:
pq.read_metadata("../data/intermediate/sph_2000_county.parquet")

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 3
  num_rows: 1183644
  num_row_groups: 2
  format_version: 2.6
  serialized_size: 2547

In [5]:
# Open the parquet file
parquet_file = pq.ParquetFile('../data/intermediate/sph_2000_county.parquet')

# Read the first 10 rows (you can adjust this as needed)
num_rows = 10
table = parquet_file.read_row_group(0).slice(0, num_rows)

# Convert to pandas DataFrame if needed
df = table.to_pandas()

# Display the data
print(df.head(num_rows))

              day       sph
county                     
01001  2000-01-01  0.008447
01009  2000-01-01  0.007666
01023  2000-01-01  0.007780
01033  2000-01-01  0.007690
01049  2000-01-01  0.007920
01089  2000-01-01  0.007715
01109  2000-01-01  0.008698
01133  2000-01-01  0.007672
02016  2000-01-01       NaN
02090  2000-01-01       NaN


In [7]:
conn.execute("CREATE TABLE gridmet AS SELECT county, day, sph FROM '../data/intermediate/sph_2000_county.parquet'")
conn.execute("SELECT * FROM gridmet LIMIT 10").fetchdf()

,county,day,sph
0,01001,2000-01-01,0.008447
1,01009,2000-01-01,0.007666
2,01023,2000-01-01,0.007780
3,01033,2000-01-01,0.007690
4,01049,2000-01-01,0.007920
5,01089,2000-01-01,0.007715
6,01109,2000-01-01,0.008698
7,01133,2000-01-01,0.007672
8,02016,2000-01-01,NaN
9,02090,2000-01-01,NaN


In [8]:
conn.execute("""
    SELECT 
        COUNT(DISTINCT county) AS n, 
    FROM 
        '../data/intermediate/sph_2000_county.parquet' 
    WHERE
        sph is NULL
""").fetchdf()

,n
0,125


In [16]:
conn.execute("""
    SELECT 
        * 
    FROM 
        '../data/intermediate/sph_2020_zcta.parquet' 
    WHERE
        day = '2020-01-01'
""").fetchdf()

,day,sph,zcta
0,2020-01-01,0.002679,15301
1,2020-01-01,0.002483,15658
2,2020-01-01,0.002613,15601
3,2020-01-01,0.002301,17720
4,2020-01-01,0.002073,18843
...,...,...,...
33786,2020-01-01,0.002663,79837
33787,2020-01-01,0.002417,88338
33788,2020-01-01,0.003044,40475
33789,2020-01-01,NaN,99738


In [18]:
conn.execute("""
    SELECT 
        * 
    FROM 
        '../data/output/gridmet_2020.parquet' 
    WHERE
        date = '2020-01-01' AND zcta = '15301'
""").fetchdf()

,zcta,date,sph,vpd,tmmn,tmmx,pr,rmin,rmax,srad,vs,th
0,15301,2020-01-01,0.002679,0.248977,270.409271,277.549957,0.0,48.831684,76.600449,83.361565,6.102479,237.928329


In [21]:
conn.execute("""
    SELECT 
        * 
    FROM 
        '../data/output/gridmet_2020.parquet' 
    LIMIT 10
""").fetchdf()

,zcta,date,sph,vpd,tmmn,tmmx,pr,rmin,rmax,srad,vs,th
0,01001,2020-01-01,0.002286,0.315761,271.296967,277.555237,0.0,44.178677,64.760246,48.408112,5.303896,268.844574
1,01002,2020-01-01,0.002316,0.270970,270.678040,276.403229,0.0,48.807545,67.122864,48.690861,6.194613,277.045380
2,01003,2020-01-01,0.002324,0.275687,270.570160,276.721832,0.0,48.330811,68.625679,47.801319,6.621912,279.931610
3,01005,2020-01-01,0.002278,0.254371,270.060822,275.801392,0.0,49.648422,68.018799,54.278908,4.757792,271.595367
4,01007,2020-01-01,0.002292,0.294850,271.408905,276.611603,0.0,47.321125,62.962490,50.229713,5.398530,272.681793
5,01008,2020-01-01,0.002170,0.196460,268.538025,273.645081,0.0,54.349739,72.103798,43.948208,6.059861,272.166107
6,01009,2020-01-01,0.002278,0.311203,271.712982,276.978851,0.0,45.789577,61.664722,51.935951,5.105854,268.369110
7,01010,2020-01-01,0.002260,0.304169,271.530243,276.645782,0.0,45.776703,61.463512,54.338177,5.586394,268.541901
8,01011,2020-01-01,0.002223,0.192180,268.435944,273.895630,0.0,54.887627,73.973236,41.679115,6.195394,271.681152
9,01012,2020-01-01,0.002216,0.198764,268.080536,274.410370,0.0,53.074966,76.328491,43.083988,6.112026,273.795654


In [2]:
conn.execute("""
    SELECT 
        count(zcta) AS nrows 
    FROM 
        '../data/output/gridmet_2020.parquet' 
""").fetchdf()

,nrows
0,12184140


In [3]:
conn.execute("""
    SELECT 
        count(distinct zcta) AS n_zcta, 
        count(distinct date) AS n_dates
    FROM 
        '../data/output/gridmet_2020.parquet' 
""").fetchdf()

,n_zcta,n_dates
0,33290,366
